In [ ]:
!nvidia-smi #Check GPUs requires atleast 24GB of memory 

In [ ]:
!apt-get update
!apt-get install ninja-build #Installing Ninja (Not neccessary if you already have it installed))

In [ ]:
!pip install torch==1.12.0+cu116 torchvision==0.13.0+cu116 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu116 #Installing Torch to compatible version

In [ ]:
!pip install click

In [ ]:
!pip install scipy

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.6.0/local_installers/cuda-repo-ubuntu1804-11-6-local_11.6.0-510.39.01-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-11-6-local_11.6.0-510.39.01-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu1804-11-6-local/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda

#Installs CUDA

In [ ]:
!sudo apt-get update && sudo apt-get install -y build-essential
#Install essential files (Run !nvidia-smi to see if GPU drivers are still installed. If not, restart instance or machine)

In [ ]:
!ls /usr/local/cuda-11.6/lib64 | grep cusolver #Locating cusolver if exists

In [ ]:
!find /usr -iname libcudart.so* #Locating cusolver if exists

In [ ]:
#On VM, the machine couldn't locate the right path to CUDA so I had to link it manually. This step isn't required unless training.py doesn't detect CUDA to start up its plugins with
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-11.6/lib64
!export CUDA_HOME=/usr/local/cuda-11.6
import os

cuda_root = '/usr/local/cuda-11.6'
if os.path.exists(cuda_root):
    os.environ['CUDA_HOME'] = cuda_root
    os.environ['LD_LIBRARY_PATH'] = f"{os.environ['LD_LIBRARY_PATH']}:{cuda_root}/targets/x86_64-linux/lib"
    if 'LIBRARY_PATH' in os.environ:
        os.environ['LIBRARY_PATH'] = f"{os.environ['LIBRARY_PATH']}:{cuda_root}/targets/x86_64-linux/lib"
    else:
        os.environ['LIBRARY_PATH'] = f"{cuda_root}/targets/x86_64-linux/lib"
    os.environ['PATH'] = f"{os.environ['PATH']}:{cuda_root}/bin"


In [ ]:
#Editing grid_sample file as there is a tuple error (Try without as only the error occured at the pre-training stage)
content = """
# Copyright (c) 2021, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.
#
# NVIDIA CORPORATION and its licensors retain all intellectual property
# and proprietary rights in and to this software, related documentation
# and any modifications thereto.  Any use, reproduction, disclosure or
# distribution of this software and related documentation without an express
# license agreement from NVIDIA CORPORATION is strictly prohibited.

#Custom replacement for `torch.nn.functional.grid_sample` that
#supports arbitrarily high order gradients between the input and output.
#Only works on 2D images and assumes
#`mode='bilinear'`, `padding_mode='zeros'`, `align_corners=False`.

import torch
from pkg_resources import parse_version

# pylint: disable=redefined-builtin
# pylint: disable=arguments-differ
# pylint: disable=protected-access

#----------------------------------------------------------------------------

enabled = False  # Enable the custom op by setting this to true.
_use_pytorch_1_11_api = parse_version(torch.__version__) >= parse_version('1.11.0a') # Allow prerelease builds of 1.11

#----------------------------------------------------------------------------

def grid_sample(input, grid):
    if _should_use_custom_op():
        return _GridSample2dForward.apply(input, grid)
    return torch.nn.functional.grid_sample(input=input, grid=grid, mode='bilinear', padding_mode='zeros', align_corners=False)

#----------------------------------------------------------------------------

def _should_use_custom_op():
    return enabled

#----------------------------------------------------------------------------

class _GridSample2dForward(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, grid):
        assert input.ndim == 4
        assert grid.ndim == 4
        output = torch.nn.functional.grid_sample(input=input, grid=grid, mode='bilinear', padding_mode='zeros', align_corners=False)
        ctx.save_for_backward(input, grid)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, grid = ctx.saved_tensors
        grad_input, grad_grid = _GridSample2dBackward.apply(grad_output, input, grid)
        return grad_input, grad_grid

#----------------------------------------------------------------------------

class _GridSample2dBackward(torch.autograd.Function):
    @staticmethod
    def forward(ctx, grad_output, input, grid):
        op, _ = torch._C._jit_get_operation('aten::grid_sampler_2d_backward')
        if _use_pytorch_1_11_api:
            output_mask = (ctx.needs_input_grad[1], ctx.needs_input_grad[2])
            grad_input, grad_grid = op(grad_output, input, grid, 0, 0, False, output_mask)
        else:
            grad_input, grad_grid = op(grad_output, input, grid, 0, 0, False)
        ctx.save_for_backward(grid)
        return grad_input, grad_grid

    @staticmethod
    def backward(ctx, grad2_grad_input, grad2_grad_grid):
        _ = grad2_grad_grid # unused
        grid, = ctx.saved_tensors
        grad2_grad_output = None
        grad2_input = None
        grad2_grid = None

        if ctx.needs_input_grad[0]:
            grad2_grad_output = _GridSample2dForward.apply(grad2_grad_input, grid)

        assert not ctx.needs_input_grad[2]
        return grad2_grad_output, grad2_input, grad2_grid

#----------------------------------------------------------------------------
"""

with open('stylegan3/torch_utils/ops/grid_sample_gradfix.py', 'w') as f:  #Replace with directory of grid_sample_gradfix.py
    f.write(content)




In [ ]:
#Converting Images into required format for dataset (RGB Format and 1024X1024)
from PIL import Image
import os

def get_new_size(img, target_size):
    width, height = img.size
    aspect_ratio = width / height

    if width > height:
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:
        new_width = int(target_size * aspect_ratio)
        new_height = target_size

    return new_width, new_height

def pad_image(img, target_size, padding_color):
    width, height = img.size
    result = Image.new(img.mode, (target_size, target_size), padding_color)
    x = (target_size - width) // 2
    y = (target_size - height) // 2
    result.paste(img, (x, y))
    return result

input_directory = './your_destination_folder/resized/necklacesbatch5/home_your_destination_folder_resized_rgb_necklace' #Input folder of images
output_directory = 'Necklaces_Output_5' #Output folder

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

padding_color = (255, 255, 255, 255)  # Change this to the desired padding color (R, G, B, A)

for filename in os.listdir(input_directory):
    input_filepath = os.path.join(input_directory, filename)
    output_filepath = os.path.join(output_directory, os.path.splitext(filename)[0] + '.png')

    try:
        with Image.open(input_filepath) as img:
            img = img.convert('RGB')  # Convert to PNG format
            new_size = get_new_size(img, 1024)
            img = img.resize(new_size, Image.LANCZOS)  # Resize while maintaining aspect ratio
            img = pad_image(img, 1024, padding_color)  # Pad the image to make it 1024x1024
            img.save(output_filepath, 'PNG')
    except Exception as e:
        print(f"Error processing {filename}: {e}")



In [ ]:
#Processing the dataset

!python dataset_tool.py --source=./resized/necklaces --dest=data/datanecklaces


  0%|                                                   | 0/516 [00:00<?, ?it/s]/home/your_destination_folder/dataset_tool.py:229: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((ww, hh), PIL.Image.LANCZOS)
100%|█████████████████████████████████████████| 516/516 [00:23<00:00, 22.03it/s]


In [ ]:
!python train.py --outdir=outputs/necklaces-output --cfg=stylegan3-t --data=data/datanecklaces --gpus=4 --batch=8  --batch-gpu=2 --mbstd-group=2 --gamma=96 --snap=10 --mirror=1 --aug=ada --metrics=none --resume=./network-snapshot-000520.pkl
#Training script resuming from previous snapshot
#Replace no. of gpus and make sure --batch-gpu and --mbstd-group is mathematically correct with the no. of gpus and batch size.
#Example: Batch = 8, GPUS = 4, each gpu will take a batch (--batch-gpu) of 2.

#--snap will determine at every 10 ticks, the network will save a snapshot of itself.
#replace --resume with a specific network snapshot.

In [ ]:
#This code allows generation of the images using the network snapshot. 
#--trunc determines how experimental it could go from values 0-2
#Network should be the snapshot you want to train from.

!python gen_images.py --outdir=out --trunc=0.7 --network=./network-snapshot-000520.pkl


Loading networks from "./outputs/necklaces-output/00013-stylegan3-t-necklace-gpus4-batch8-gamma64/network-snapshot-000880.pkl"...
Generating image for seed 0 (0/100) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
Generating image for seed 1 (1/100) ...
Generating image for seed 2 (2/100) ...
Generating image for seed 3 (3/100) ...
Generating image for seed 4 (4/100) ...
Generating image for seed 5 (5/100) ...
Generating image for seed 6 (6/100) ...
Generating image for seed 7 (7/100) ...
Generating image for seed 8 (8/100) ...
Generating image for seed 9 (9/100) ...
Generating image for seed 10 (10/100) ...
Generating image for seed 11 (11/100) ...
Generating image for seed 12 (12/100) ...
Generating image for seed 13 (13/100) ...
Generating image for seed 14 (14/100) ...
Generating image for seed 15 (15/100) ...
Generating image for seed 16 (16/100) ...
Generating image for seed 17 (17/100) ...
Generating image for s

In [ ]:
#If the gen_images.py cannot locate plugins
import os

cuda_root = '/usr/local/cuda-11.6'
if os.path.exists(cuda_root):
    os.environ['CUDA_HOME'] = cuda_root
    os.environ['LD_LIBRARY_PATH'] = f"{os.environ['LD_LIBRARY_PATH']}:{cuda_root}/targets/x86_64-linux/lib"
    if 'LIBRARY_PATH' in os.environ:
        os.environ['LIBRARY_PATH'] = f"{os.environ['LIBRARY_PATH']}:{cuda_root}/targets/x86_64-linux/lib"
    else:
        os.environ['LIBRARY_PATH'] = f"{cuda_root}/targets/x86_64-linux/lib"
    os.environ['CPATH'] = f"{cuda_root}/include:{os.environ.get('CPATH', '')}"
    os.environ['PATH'] = f"{os.environ['PATH']}:{cuda_root}/bin"
